## Autoreload

In [1]:
%load_ext autoreload
%autoreload 2

## Necessary Imports

In [2]:
import os
import numpy as np
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
verbosity = 2
if verbosity > 0:
    print(f"Torch {torch.__version__}")
    if torch.cuda.is_available():
        print("CUDA available")
    else:
        print("CUDA unavailable")
    print(f"Device: {device}")
import sys
REPO_DIR = '/media/kristijan/kristijan-hdd-ex/ShapeFromImages/'
sys.path.append(REPO_DIR)

from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor

from models.smpl_official import SMPL

import config

from data.synthetic_training_dataset import SyntheticTrainingDataset

from renderers.nmr_renderer import NMRRenderer

from utils.cam_utils import get_intrinsics_matrix

/media/kristijan/kristijan-hdd-ex/ShapeFromImages/.venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch 1.4.0
CUDA available
Device: cuda:0


## Constants

In [3]:
resnet_in_channels = 1 + 17  # single-channel silhouette + 17 joint heatmaps
resnet_layers = 18
ief_iters = 3
print("\nResNet in channels:", resnet_in_channels)
print("ResNet layers:", resnet_layers)
print("IEF Num iters:", ief_iters)

batch_size = 1


ResNet in channels: 18
ResNet layers: 18
IEF Num iters: 3


## Dataset Paths

In [7]:
# ----------------------- Paths -----------------------
# Path to npz with training data.
#train_path = 'data/amass_up3d_3dpw_train.npz'
# Path to npz with validation data.
#val_path = 'data/up3d_3dpw_val.npz'

## Dataset
Not needed because I have fixed poses.

In [8]:
#train_dataset = SyntheticTrainingDataset(npz_path=train_path, params_from='all')
#val_dataset = SyntheticTrainingDataset(npz_path=val_path, params_from='all')
#train_val_monitor_datasets = [train_dataset, val_dataset]
#print("Training examples found:", len(train_dataset))
#print("Validation examples found:", len(val_dataset))

FileNotFoundError: [Errno 2] No such file or directory: 'data/amass_up3d_3dpw_train.npz'

## SMPL Model
NOTE: Run this only once, didn't figure out why.

In [4]:
smpl_model_dir = os.path.join(REPO_DIR, config.SMPL_MODEL_DIR)

smpl_model = SMPL(smpl_model_dir, num_betas=10)

<class 'models.smpl_official.SMPL'>


## Create Renderer

In [7]:
# Camera and NMR part/silhouette renderer
# Assuming camera rotation is identity (since it is dealt with by global_orients in SMPL)
mean_cam_t = np.array([0., 0.2, 42.])
mean_cam_t = torch.from_numpy(mean_cam_t).float().to(device)
mean_cam_t = mean_cam_t[None, :].expand(batch_size, -1)
cam_K = get_intrinsics_matrix(config.REGRESSOR_IMG_WH, config.REGRESSOR_IMG_WH, config.FOCAL_LENGTH)
cam_K = torch.from_numpy(cam_K.astype(np.float32)).to(device)
cam_K = cam_K[None, :, :].expand(batch_size, -1, -1)
cam_R = torch.eye(3).to(device)
cam_R = cam_R[None, :, :].expand(batch_size, -1, -1)
nmr_parts_renderer = NMRRenderer(batch_size,
                                 cam_K,
                                 cam_R,
                                 config.REGRESSOR_IMG_WH,
                                 rend_parts_seg=True)

smpl_model.to(device)
nmr_parts_renderer.to(device)

NMRRenderer(
  (renderer): Renderer()
)

## Augmentation

In [8]:
# ----------------------- Augmentation -----------------------
# SMPL
augment_shape = True
delta_betas_distribution = 'normal'
delta_betas_std_vector = torch.tensor([1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5],
                                      device=device).float()  # used if delta_betas_distribution is 'normal'
delta_betas_range = [-3., 3.]  # used if delta_betas_distribution is 'uniform'
smpl_augment_params = {'augment_shape': augment_shape,
                       'delta_betas_distribution': delta_betas_distribution,
                       'delta_betas_std_vector': delta_betas_std_vector,
                       'delta_betas_range': delta_betas_range}
# Camera
xy_std = 0.05
delta_z_range = [-5, 5]
cam_augment_params = {'xy_std': xy_std,
                      'delta_z_range': delta_z_range}
# BBox
crop_input = True
mean_scale_factor = 1.2
delta_scale_range = [-0.2, 0.2]
delta_centre_range = [-5, 5]
bbox_augment_params = {'crop_input': crop_input,
                       'mean_scale_factor': mean_scale_factor,
                       'delta_scale_range': delta_scale_range,
                       'delta_centre_range': delta_centre_range}